In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt
import re
import pandas as pd
import numpy as np

# Import data

In [ ]:
folder_path = "xxxx"

## Liste des comptes

In [ ]:
df_liste_comptes = pd.read_excel(folder_path + "/raw_data/0. Data Releve bancaire/Bank accounts mapping.xlsx", sheet_name = "Synthèse globale",header=1)

## Relevé

In [ ]:
df_credit = pd.read_excel(folder_path + "/transformed_data/2. Banque/All_Lignes de crédit.xlsx",sheet_name="Sheet1")

In [ ]:
df_credit_old = pd.read_excel(folder_path + "/raw_data/6. Données modifiées manuellement/1. Relevé_ancienne version/All_lignes de crédit.xlsx",sheet_name="crédit")

### Données déjà traitées

#### Virements

In [ ]:
###### Résultat du rapprochement des virements réalisés par l'algo
### Il s'agit normalement du résultat de sortie de la Notebook 1.1. Mais dans cette version, je préfère d'utiliser la version déjà en prod pour éviter qu'on vient de lancer la Notebook 1.1 pour un petit test
### Et surtout il vaut mieux de fixer une version en prod au lieu de changer la version tout le temps
df_data_validee_DM = pd.read_excel(folder_path + "/raw_data/1. Virements/Données de rapprochement validées/All_Démembrement validé.xlsx",sheet_name="Démembrement")
df_data_validee_PP = pd.read_excel(folder_path + "/raw_data/1. Virements/Données de rapprochement validées/All_Pleine propriété validée.xlsx",sheet_name="PP")

In [ ]:
df_heavy_checks = pd.read_excel(folder_path + "/raw_data/7. Taskforce/All_Données à vérifier manuellement_heavy checks_V16 février.xlsx",sheet_name="Données_non_rapprochées", header=2)
df_propositions = pd.read_excel(folder_path + "/raw_data/7. Taskforce/All_Données à vérifier manuellement_proposition.xlsx",sheet_name="Données_non_rapprochées", header=2)

# Data cleaning

## Liste des comptes

In [ ]:
df_liste_comptes = df_liste_comptes[['Titulaire', 'IBAN', 'Collecte', 'Pays']]
df_liste_comptes = df_liste_comptes[~df_liste_comptes["IBAN"].isin(["IBAN","Voir avec fund Manager",np.nan])]
df_liste_comptes["IBAN"] = df_liste_comptes["IBAN"].str.strip().str.replace("EUR","").str.replace("-"," ").str.strip()

## Chercher les lignes Autres

In [ ]:
df_data_validee = pd.concat([df_data_validee_DM,df_data_validee_PP])
df_data_validee = df_data_validee[["Id sys"]]#.rename(columns={"Fonds_relevé":"Fonds"})

df_data_validee["Id sys"] = df_data_validee["Id sys"].str.split("|")
df_data_validee = df_data_validee.explode("Id sys")

In [ ]:
df_check_manuel = pd.concat([df_heavy_checks,df_propositions])
df_check_manuel = df_check_manuel[["Id sys"]]

In [ ]:
df_virements_traites = pd.concat([df_data_validee,df_check_manuel]).rename(columns={"Id sys":"Id sys_old"})

In [ ]:
df_merge = df_credit.merge(df_virements_traites,on=["Id sys_old"],how="outer",indicator=True)
df_donnees_restantes = df_merge[df_merge["_merge"]=="left_only"].drop(columns="_merge")

In [ ]:
### Enlever les données de la Bred qui sont lancées après la version prod déjà validée
mask = df_donnees_restantes["Id sys_old"]=="Nouvelles_données_intégrées"
df_nouvelles_donnees = df_donnees_restantes.loc[mask,:]
df_donnees_restantes = df_donnees_restantes[~df_donnees_restantes["Id sys_old"].isin(df_nouvelles_donnees["Id sys_old"])]

In [ ]:
df_donnees_restantes["Type_Transaction"].unique()

In [ ]:
df_donnees_autres = df_donnees_restantes[~df_donnees_restantes["Type_Transaction"].str.contains("REMISE|PRLV|REJET",regex=True,na=False)]

In [ ]:
df_donnees_autres.to_excel(folder_path+"/output/4. Autres/Données restantes à checker.xlsx",sheet_name="données restantes",index=False)

__Ces lignes doivent être ensuite vérifiées par un humain, car il faut avoir le jugement humain pour les reclasser.
Une version déjà traitée par Merlin.__

## Virements CAC

In [ ]:
df_heavy_checks["Collecte ou pas collecte ?"].unique()

In [ ]:
df_heavy_checks_cac = df_heavy_checks[df_heavy_checks["Collecte ou pas collecte ?"]!="Non, le virement reçu est bien de la collecte"]

In [ ]:
len(df_heavy_checks[df_heavy_checks["Collecte ou pas collecte ?"]=="Peut-être : j'ai un doute, le virement reçu n'est peut-être pas de la collecte"])

In [ ]:
df_heavy_checks["Collecte ou pas collecte ?"].value_counts()

In [ ]:
#### Importer les données déjà vérifiées manuellement #######
### Il s'agit du résultat de la partie juste au-dessus (Chercher les lignes Autres) mais déjà traitées par Merlin
df_cac = pd.read_excel(folder_path+"/raw_data/6. Données modifiées manuellement/2. Virement CAC/CAC_Données restantes à checker manuellement.xlsx",sheet_name="Analyse_données restantes")

In [ ]:
patern_titulaire = r'(?<=FRM)\s+([a-zA-Z\s\-\.\d]*)|(?<=DE)\s+([a-zA-Z\s\-\.\d]*)|(?<=NAME)\/([a-zA-Z\s\-\.\d]*)'
df_result = df_cac.loc[:,'Libellé'].str.extract(patern_titulaire)
df_cac.loc[:,"Titulaire"] = df_result[0].combine_first(df_result[1]).combine_first(df_result[2])
df_cac.loc[:,"Titulaire"] = df_cac.loc[:,"Titulaire"].str.strip()

In [ ]:
df_cac = df_cac.rename(columns={"Fonds":"Compte_récepteur","IBAN":"IBAN_récepteur",
                                             "Titulaire":"Compte_emetteur"})

In [ ]:
df_liste_comptes = df_liste_comptes.rename(columns={"Titulaire":"Titulaire_récepteur","IBAN":"IBAN_récepteur",
                                                    "Collecte":"Compte_récepteur est de la collecte?","Pays":"Compte_récepteur_pays"})

In [ ]:
df_cac = df_cac.merge(df_liste_comptes, on="IBAN_récepteur")

In [ ]:
df_cac_collecte = df_cac[df_cac["Vérification manuelle Merlin"]=="CAC: Régularisation de collecte"]
df_cac_stat = df_cac_collecte.groupby(by=["Compte_récepteur","IBAN_récepteur","Compte_emetteur"])["Crédit"].sum().reset_index().sort_values(by=["Compte_récepteur","Crédit"],ascending=False)


In [ ]:
df_cac.to_excel(folder_path+"/output/4. Autres/Virements CAC_details.xlsx",sheet_name="CAC",index=False)

In [ ]:
df_cac_stat.to_excel(folder_path+"/output/4. Autres/Virements CAC_stats.xlsx",sheet_name="CAC",index=False)